This code is used to combine feature maps all channles together

In [5]:
from Model import Model
from tensorboardX import SummaryWriter
from torchvision import transforms
import torch
image_path = './example_images/000186.jpg'
from PIL import Image
import cv2
import numpy as np
from torch.autograd import Variable
import torchvision.utils as vutils

In [6]:
net = Model()
net.load_state_dict(torch.load('weights/epoch204densenet.pth'))
net.eval()

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplac

In [7]:
def preprocess_image(cv2im, resize_im=True):
    """
        Processes image for CNNs

    Args:
        PIL_img (PIL_img): Image to process
        resize_im (bool): Resize to 224 or not
    returns:
        im_as_var (Pytorch variable): Variable that contains processed float tensor
    """
    # mean and std list for channels (Imagenet)
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    # Resize image
    if resize_im:
        cv2im = cv2.resize(cv2im, (224, 224))
    im_as_arr = np.float32(cv2im)
    im_as_arr = np.ascontiguousarray(im_as_arr[..., ::-1])
    im_as_arr = im_as_arr.transpose(2, 0, 1)  # Convert array to D,W,H
    # Normalize the channels
    for channel, _ in enumerate(im_as_arr):
        im_as_arr[channel] /= 255
        im_as_arr[channel] -= mean[channel]
        im_as_arr[channel] /= std[channel]
    # Convert to float tensor
    im_as_ten = torch.from_numpy(im_as_arr).float()
    # Add one more channel to the beginning. Tensor shape = 1,3,224,224
    im_as_ten.unsqueeze_(0)
    # Convert to Pytorch variable
    im_as_var = Variable(im_as_ten, requires_grad=True)
    return im_as_var

In [8]:
class FeatureVisualization():
    def __init__(self,img_path,selected_layer):
        self.img_path=img_path
        self.selected_layer=selected_layer
        self.pretrained_model = net.features 
        
    def process_image(self):
        img=cv2.imread(self.img_path)
        print(img.shape)
        self.height, self.width, _ = img.shape
        print('22222', self.height, self.width)
        img=preprocess_image(img)

        return img

    def get_feature(self):
        # input = Variable(torch.randn(1, 3, 224, 224))
        #writer = SummaryWriter()
        input=self.process_image()
        print(input.shape)
        x=input
        for index,layer in enumerate(self.pretrained_model):
            #print('index',index, layer)
            x=layer(x)
            if (index == self.selected_layer):
                img_grid = vutils.make_grid(x, normalize=True, scale_each=True, nrow=2)  # B，C, H, W
                #writer.add_image('_feature_maps', img_grid, global_step=666)
                return x

    def get_allfeature(self):
        features=self.get_feature()
        bz, nc, h, w = features.shape
        print('features shape',features.shape)
        features = torch.squeeze(features)
        print(features.shape)
        features = torch.sum(features, 0)
        print(features)
        features = features - torch.min(features)
        features = features / torch.max(features)
        print(features)
        #feature=features.reshape()

        return features

    def save_feature_to_img(self):
        #to numpy
        feature=self.get_allfeature()
        feature=feature.data.numpy()
        
        #use sigmod to [0,1]
        #feature= 1.0/(1+np.exp(-1*feature))

        # to [0,255]
        feature=np.round(feature*255)
        feature = cv2.resize(feature, (self.width,self.height))
        feature = feature.astype(np.uint8)
        #feature = cv2.applyColorMap(feature, cv2.COLORMAP_AUTUMN)

        cv2.imwrite('./feature/all/img_all.jpg', feature)


In [9]:
if __name__=='__main__':
    # get class
    myClass=FeatureVisualization(image_path,11)
    print (myClass.pretrained_model)

    myClass.save_feature_to_img()

Sequential(
  (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu0): ReLU(inplace)
  (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (denseblock1): _DenseBlock(
    (denselayer1): _DenseLayer(
      (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU(inplace)
      (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu2): ReLU(inplace)
      (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    )
    (denselayer2): _DenseLayer(
      (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU(inplace)
      (conv1): Conv2d(96, 128, kernel_size=(1, 1), st

features shape torch.Size([1, 1024, 7, 7])
torch.Size([1024, 7, 7])
tensor([[114.8183, 123.3362,  80.8103, -47.9569, -53.5997, -28.9836, -22.6118],
        [241.4968, 259.7883, 240.4349,  65.4001, -40.7814, -30.6308, -13.1906],
        [258.8985, 366.0671, 435.1502, 273.4243, 127.6782,   1.1409, -24.4903],
        [255.6081, 496.6424, 662.5732, 509.9329, 295.2365,  14.0489, -38.8309],
        [191.3058, 441.8145, 682.6691, 597.9196, 407.4054,  55.9122, -30.1649],
        [174.3336, 392.1590, 634.9929, 577.1724, 441.0374,  83.5178,  -7.2569],
        [ 41.3197, 141.5540, 284.7692, 313.0724, 265.6495,  63.4043, -11.1152]],
       grad_fn=<SumBackward2>)
tensor([[0.2287, 0.2403, 0.1826, 0.0077, 0.0000, 0.0334, 0.0421],
        [0.4008, 0.4256, 0.3994, 0.1616, 0.0174, 0.0312, 0.0549],
        [0.4244, 0.5700, 0.6638, 0.4442, 0.2462, 0.0743, 0.0395],
        [0.4200, 0.7473, 0.9727, 0.7654, 0.4738, 0.0919, 0.0201],
        [0.3326, 0.6729, 1.0000, 0.8849, 0.6261, 0.1487, 0.0318],
        [0